In [1]:
!pip install nbimporter

In [2]:
import sqlite3
import pandas as pd
import nbimporter
from final_scores import calculate_final_driver_places 

conn = sqlite3.connect('formula1.db')

drivers_final_positions = calculate_final_driver_places(conn)

print(drivers_final_positions)

      year  driverId      driver_name  points  place
73    1950       642      Nino Farina    30.0      1
66    1950       579      Juan Fangio    27.0      2
72    1950       786    Luigi Fagioli    24.0      3
70    1950       627     Louis Rosier    13.0      4
56    1950       647   Alberto Ascari    11.0      5
...    ...       ...              ...     ...    ...
3133  2022       855      Guanyu Zhou     6.0     18
3136  2022       848  Alexander Albon     4.0     19
3139  2022       849  Nicholas Latifi     2.0     20
3145  2022       856    Nyck de Vries     2.0     20
3140  2022       807  Nico Hülkenberg     0.0     22

[3146 rows x 5 columns]


In [10]:
driver_participation = drivers_final_positions.groupby(['driverId', 'year']).size().reset_index(name='participated')

driver_seasons = driver_participation[driver_participation['participated'] > 0][['driverId', 'year']]

driver_years_list = driver_seasons.groupby('driverId')['year'].apply(list).reset_index(name='seasons_participated')

driver_years_list['number_of_seasons'] = driver_years_list['seasons_participated'].apply(len)

print(driver_years_list)

csv_file_path_reshaped = 'number_of_seasons.csv'
driver_years_list.to_csv(csv_file_path_reshaped, index=False)

     driverId                               seasons_participated  \
0           1  [2007, 2008, 2009, 2010, 2011, 2012, 2013, 201...   
1           2  [2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...   
2           3  [2006, 2007, 2008, 2009, 2010, 2011, 2012, 201...   
3           4  [2001, 2003, 2004, 2005, 2006, 2007, 2008, 200...   
4           5         [2007, 2008, 2009, 2010, 2011, 2012, 2013]   
..        ...                                                ...   
843       852                                       [2021, 2022]   
844       853                                             [2021]   
845       854                                       [2021, 2022]   
846       855                                             [2022]   
847       856                                             [2022]   

     number_of_seasons  
0                   16  
1                   12  
2                   11  
3                   19  
4                    7  
..                 ...  
843     

In [11]:
import json
import sqlite3
import pandas as pd

# Function to calculate lap positions
def calculate_lap_positions(conn):
    # Your SQL query
    query = '''
    WITH RankedPositions AS (
        SELECT
            d.driverId,
            r.year,
            lt.position AS position,
            COUNT(lt.position) AS position_count,
            ROW_NUMBER() OVER (PARTITION BY d.driverId, r.year ORDER BY COUNT(lt.position) DESC) AS position_rank,
            COUNT(lt.position) OVER (PARTITION BY d.driverId, r.year) AS total_races
        FROM
            drivers d
        JOIN
            lap_times lt ON d.driverId = lt.driverId
        JOIN
            races r ON lt.raceId = r.raceId
        GROUP BY
            d.driverId, r.year, lt.position
    )
    SELECT
        driverId,
        year,
        position,
        position_count
    FROM
        RankedPositions
    WHERE
        position_rank <= 3
    '''

    # Execute the SQL query and read the result into a DataFrame
    df = pd.read_sql_query(query, conn)

    # Group by driverId and aggregate the lap positions into a list for each driver
    lap_positions_json_df = df.groupby('driverId').apply(
        lambda group: json.dumps({year: group[['position', 'position_count']].to_dict(orient='records') for year, group in group.groupby('year')})
    ).reset_index(name='lap_positions')

    return lap_positions_json_df

# Function to calculate best and worst seasons
def calculate_best_worst_seasons(drivers_final_positions, driver_years_list, lap_positions_json_df):
    # Merge the lap positions DataFrame with the 'drivers_final_positions' DataFrame
    drivers_final_positions = pd.merge(drivers_final_positions, lap_positions_json_df, on='driverId', how='left')

    # Fill NaN values with an empty list
    drivers_final_positions['lap_positions'].fillna(value='[]', inplace=True)

    # Merge with the 'driver_years_list' DataFrame to get the number of seasons
    drivers_final_positions = pd.merge(drivers_final_positions, driver_years_list[['driverId', 'number_of_seasons']], on='driverId', how='left')

    # Identify best and worst seasons based on total points
    best_seasons = drivers_final_positions.loc[drivers_final_positions.groupby('driverId')['points'].idxmax()]
    worst_seasons = drivers_final_positions.loc[drivers_final_positions.groupby('driverId')['points'].idxmin()]

    return best_seasons, worst_seasons

# Connect to the database
conn = sqlite3.connect('formula1-originalDB.db')

# Call the functions to calculate lap positions and best/worst seasons
lap_positions_json_df = calculate_lap_positions(conn)
best_seasons, worst_seasons = calculate_best_worst_seasons(drivers_final_positions, driver_years_list, lap_positions_json_df)

# Display the results
print("Best Seasons:")
print(best_seasons[['driverId', 'year', 'points', 'best_position', 'lap_positions', 'number_of_seasons']])

print("\nWorst Seasons:")
print(worst_seasons[['driverId', 'year', 'points', 'worst_position', 'lap_positions', 'number_of_seasons']])


Best Seasons:


KeyError: "['best_position'] not in index"